In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [54]:
url_inicial = 'https://rppoblacion.uaemex.mx/issue/archive?issuesPage=5#issues'
url_root = 'https://rppoblacion.uaemex.mx/issue/archive?issuesPage=5#issues'
r=requests.get(url_inicial)

In [55]:
soup = BeautifulSoup(r.text, 'html.parser')

In [56]:
box = soup.find('div', class_='issues media-list')
volumen=box.findAll('div', class_='issue-summary media')
vol = [x.find('a').get('href')for x in volumen]
vol

['https://rppoblacion.uaemex.mx/issue/view/941',
 'https://rppoblacion.uaemex.mx/issue/view/942',
 'https://rppoblacion.uaemex.mx/issue/view/956',
 'https://rppoblacion.uaemex.mx/issue/view/957',
 'https://rppoblacion.uaemex.mx/issue/view/960',
 'https://rppoblacion.uaemex.mx/issue/view/966',
 'https://rppoblacion.uaemex.mx/issue/view/968',
 'https://rppoblacion.uaemex.mx/issue/view/969']

In [57]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='media-list') 
    volumen1=box1.findAll('div', class_='article-summary media article-summary-lista')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [58]:
vol2

['https://rppoblacion.uaemex.mx/article/view/18437',
 'https://rppoblacion.uaemex.mx/article/view/18438',
 'https://rppoblacion.uaemex.mx/article/view/18439',
 'https://rppoblacion.uaemex.mx/article/view/18440',
 'https://rppoblacion.uaemex.mx/article/view/18441',
 'https://rppoblacion.uaemex.mx/article/view/18442',
 'https://rppoblacion.uaemex.mx/article/view/18783',
 'https://rppoblacion.uaemex.mx/article/view/18586',
 'https://rppoblacion.uaemex.mx/article/view/18587',
 'https://rppoblacion.uaemex.mx/article/view/18588',
 'https://rppoblacion.uaemex.mx/article/view/18594',
 'https://rppoblacion.uaemex.mx/article/view/18595',
 'https://rppoblacion.uaemex.mx/article/view/18596',
 'https://rppoblacion.uaemex.mx/article/view/18600',
 'https://rppoblacion.uaemex.mx/article/view/18646',
 'https://rppoblacion.uaemex.mx/article/view/18669',
 'https://rppoblacion.uaemex.mx/article/view/18670',
 'https://rppoblacion.uaemex.mx/article/view/18671',
 'https://rppoblacion.uaemex.mx/article/view/1

In [59]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='media-list') 
    volumen1=box1.findAll('div', class_='article-summary media article-summary-lista')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [60]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://rppoblacion.uaemex.mx/issue/archive?issuesPage=5#issues


In [61]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

67

In [62]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [63]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Papeles de Población'
    a='V. Ciencias Sociales'
    tem='Demografía'
    s_item=BeautifulSoup(r.text,'html.parser')
 #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page-header').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='article-abstract').get_text(strip=True)
        content_book['Resumen']=resu.replace("Resumen","")
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='galley-link').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [64]:
list_scraper=list_scraper[0:236]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16
estas escrapeando la pag 17
estas escrapeando la pag 18
estas escrapeando la pag 19
estas escrapeando la pag 20
estas escrapeando la pag 21
estas escrapeando la pag 22
estas escrapeando la pag 23
estas escrapeando la pag 24
estas escrapeando la pag 25
estas escrapeando la pag 26
estas escrapeando la pag 27
estas escrapeando la pag 28
estas escrapeando la pag 29
estas escrapeando la pag 30
estas escrapeando la pag 31
estas escrapeando la pag 32
estas escrapeando la pag 33
estas escrapeando la pag 34
estas escrapeando la pag 35
es

In [65]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Papeles de Población,V. Ciencias Sociales,Demografía,La problemática teórica en los estudios de pob...,El objetivo de este trabajo es el análisis de ...,None,https://rppoblacion.uaemex.mx/article/view/18437,https://rppoblacion.uaemex.mx/article/view/184...
1,Papeles de Población,V. Ciencias Sociales,Demografía,El estudio del crecimiento de las poblaciones ...,Una población humana puede ser estudiada tanto...,None,https://rppoblacion.uaemex.mx/article/view/18438,https://rppoblacion.uaemex.mx/article/view/184...
2,Papeles de Población,V. Ciencias Sociales,Demografía,Reflexiones en torno al Programa Nacional de P...,En el Programa Nacional de Población 1995-2000...,None,https://rppoblacion.uaemex.mx/article/view/18439,https://rppoblacion.uaemex.mx/article/view/184...
3,Papeles de Población,V. Ciencias Sociales,Demografía,Algunas reflexiones sobre organización y finan...,La seguridadsocialestáhoyendíaenelmedio de una...,None,https://rppoblacion.uaemex.mx/article/view/18440,https://rppoblacion.uaemex.mx/article/view/184...
4,Papeles de Población,V. Ciencias Sociales,Demografía,La Transportación de personas en la Zona Metro...,El transporte de pasajeros en la Zona Metropol...,None,https://rppoblacion.uaemex.mx/article/view/18441,https://rppoblacion.uaemex.mx/article/view/184...
...,...,...,...,...,...,...,...,...
62,Papeles de Población,V. Ciencias Sociales,Demografía,Un tipo singular de la migración de retorno: e...,Los movimientos internacionales de población h...,None,https://rppoblacion.uaemex.mx/article/view/18962,https://rppoblacion.uaemex.mx/article/view/189...
63,Papeles de Población,V. Ciencias Sociales,Demografía,Análisis de la encuesta de trabajadores indocu...,En la planeación del desarrollo de cualquier p...,None,https://rppoblacion.uaemex.mx/article/view/18963,https://rppoblacion.uaemex.mx/article/view/189...
64,Papeles de Población,V. Ciencias Sociales,Demografía,Migración y movilidad social,La transformación incesante en que viven todas...,None,https://rppoblacion.uaemex.mx/article/view/18964,https://rppoblacion.uaemex.mx/article/view/189...
65,Papeles de Población,V. Ciencias Sociales,Demografía,Los retos del desarrollo social en la població...,"Durante los últimos años, el tema de la poblac...",None,https://rppoblacion.uaemex.mx/article/view/18965,https://rppoblacion.uaemex.mx/article/view/189...


In [66]:
df_catalogo.to_csv('Revista43.05.csv', index=False)